In [1]:
    var sparkConf = new SparkConf().setAppName("Gradient Boosting Classification");
    var sc = new SparkContext(sparkConf);


 Load and parse the data file.


In [2]:
var datapath =  "../data/mllib/sample_libsvm_data.txt";
    var data = MLUtils.loadLibSVMFile(sc, datapath);


 Split the data into training and test sets (30% held out for testing)


In [3]:
    var splits = data.randomSplit([0.7, 0.3]);
    var trainingData = splits[0];
    var testData = splits[1];


 Train a GradientBoostedTrees model.
 The defaultParams for Classification use LogLoss by default.


In [4]:
    var boostingStrategy = BoostingStrategy.defaultParams("Classification");
    boostingStrategy.setNumIterations(3); // Note: Use more iterations in practice.
    boostingStrategy.getTreeStrategy().setNumClasses(2);
    boostingStrategy.getTreeStrategy().setMaxDepth(5);


 Empty categoricalFeaturesInfo indicates all features are continuous.


In [5]:
    var categoricalFeaturesInfo = {};
    boostingStrategy.getTreeStrategy().setCategoricalFeaturesInfo(categoricalFeaturesInfo);

    var model = GradientBoostedTrees.train(trainingData, boostingStrategy);


In [6]:
    var predictionAndLabel = testData.mapToPair(function (lp, model) {
        return new Tuple(model.predict(lp.getFeatures()), lp.getLabel());
    }, [model]);


In [7]:
    var testErr = predictionAndLabel.filter(function (tuple) {
            return tuple[0] != tuple[1];
        }).count() / testData.count();

    var ret = {};
    ret.testErr = testErr;
    ret.model = model;
var result = ret;
    print("Test Error: " + result.testErr);
    print("Learned classification GBT model:\n" + result.model.toDebugString());


 Save and load model


In [8]:
    result.model.save(sc, "target/tmp/myGradientBoostingClassificationModel");
    var sameModel = GradientBoostedTreesModel.load(sc, "target/tmp/myGradientBoostingClassificationModel");

    sc.stop();
